In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import time
from matplotlib import animation
from IPython.display import HTML

Following J. Chem. Phys. 99, 1185 (1993); https://doi.org/10.1063/1.465362
$$i\frac d{dt}\left(\begin{array}{cc}\Psi_1(t)\\ \Psi_2(t)\end{array}\right) =
\left(\begin{array}{cc} 
\frac{\hat p^2}{2μ} + V_g & −D × E(t)\\ 
−D × E(t) & \frac{\hat p^2}{2μ} + V_u
\end{array}\right)
\left(\begin{array}{cc}\Psi_1(t)\\ \Psi_2(t)\end{array}\right)$$
using
$$\mathbf T=-\frac{\hbar^2}{2\mu}\left(\begin{array}{cc} 1/\text{d}x^2&0\\0&  1/\text{d}x^2\end{array}\right)$$
$$\mathbf V(t)=\left(\begin{array}{cc}V_g & −D × E(t)\\ 
−D × E(t) & V_u
\end{array}\right)$$
We get for a time step $\Delta t$:
$$\Psi(t+\Delta t) = \text{exp}\left[\Delta t \mathbf T+\Delta t \mathbf V(t+\Delta t/2)\right]\Psi(t)+\mathcal O(\Delta t^3)$$

In [3]:
m = 938.28/0.511/2
L = 30
Ngrid = 600
#x space
dx = L/(Ngrid)
x = np.linspace(dx,L,Ngrid)
_,Vg=np.loadtxt('data/H2p_pot_gerade.dat',unpack=True)
_,Vu=np.loadtxt('data/H2p_pot_ungerade.dat',unpack=True)
_,D=np.loadtxt('data/dipole_coupling.dat',unpack=True)
_,Psi0=np.loadtxt('data/H2nuclwf.dat',unpack=True)

the following functions calculate $\text{exp}\left[\lambda \mathbf T\right]$ and $\text{exp}\left[\lambda \mathbf V(t)\right]$

In [ ]:
#using fft techniques
def expDiag(lam,M):
    
def expV(lam,V):
    V1,V2=V.copy(),V.copy()
    V1[0,1]=V1[1,0]=V2[0,0]=V2[1,1]=0
    eV1=expDiag(lam/2,V1)
    

In [22]:
a=np.array([[0,1],
            [1,0]])
v,w=np.linalg.eig(a)

In [28]:
w@np.diag(v)@w.T

array([[0., 1.],
       [1., 0.]])